# July 3 2019 Firefly Demo

The best place to start is two LSST DM demo notebooks:
* Part I: https://github.com/lsst-sqre/notebook-demo/blob/master/Firefly.ipynb
* Part II: https://github.com/lsst/display_firefly/blob/master/examples/HSC-Footprints.ipynb
* Part III: Fun with HSC image differences

---
## `https://github.com/lsst-sqre/notebook-demo/blob/master/Firefly.ipynb`
# Firefly Visualization Demo

This notebook is intended to demonstrate the [Firefly](https://mospace.umsystem.edu/xmlui/handle/10355/5346) interactive interface for viewing image data. It also builds on the pedagogical explanations provided in [Getting started tutorial part 3](https://pipelines.lsst.io/getting-started/display.html) of the LSST Stack v16.0 documentation.

This tutorial seeks to teach you about how to use the LSST Science Pipelines to inspect outputs from `processCcd.py` by displaying images and source catalogs in the Firefly image viewer. In doing so, you’ll be introduced to some of the LSST Science Pipelines’ Python APIs, including:

* Accessing datasets with the `Butler`.
* Displaying images with `lsst.afw.display`
* Pass source catalog data directly to the `FireflyClient`

## Set up

This tutorial is meant to be run from the `jupyterhub` interface where the LSST stack is preinstalled. It assumes that the notebook is running a kernel with the `lsst_distrib` package set up.

We start by importing packages from the LSST stack for data access and visualization.

In [1]:
# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import numpy as np

## Creating a Butler client

All data in the LSST Pipelines flow through the `Butler`. LSST does not recommend directly accessing processed image files. Instead, use the `Butler` client available from the `lsst.daf.persistence` module imported above.

In [2]:
datadir = '/project/shared/data/Twinkles_subset/output_data_v2'
butler = Butler(datadir)

The `Butler` client reads from the data repository specified with the inputs argument. In this specific case, the data were downloaded from: [here](https://lsst-web.ncsa.illinois.edu/~krughoff/data/twinkles_subset.tar.gz).  See the `README.txt` in `/project/shared/data/Twinkles_subset` for more info.

## Listing available data IDs in the Butler

To get data from the `Butler` you need to know two things: the dataset type and the data ID.

Every dataset stored by the Butler has a well-defined type. Tasks read specific dataset types and output other specific dataset types. The `processCcd.py` command reads in raw datasets and outputs calexp, or calibrated exposure, datasets (among others). It’s calexp datasets that you’ll display in this tutorial.

Data IDs let you reference specific instances of a dataset. On the command line you select data IDs with `--id` arguments, filtering by keys like `visit`, `raft`, `ccd`, and `filter`.

Now, use the `Butler` client to find what data IDs are available for the `calexp` dataset type:



In [3]:
butler.queryMetadata('calexp', ['visit', 'raft', 'sensor'], dataId={'filter': 'r'})

[(230, '2,2', '1,1'),
 (231, '2,2', '1,1'),
 (232, '2,2', '1,1'),
 (233, '2,2', '1,1'),
 (234, '2,2', '1,1'),
 (235, '2,2', '1,1'),
 (236, '2,2', '1,1'),
 (237, '2,2', '1,1'),
 (238, '2,2', '1,1'),
 (239, '2,2', '1,1')]

The printed output is a list of `(visit, raft, ccd)` key tuples for all data IDs where the filter key is the LSST r band. 

## Get an exposure through the Butler

Knowing a specific data ID, let’s get the dataset with the `Butler` client’s get method:

In [4]:
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}
calexp = butler.get('calexp', **dataId)

The `calexp` is an `ExposureF` Python object. Exposures are powerful representations of image data because they contain not only the image data, but also a variance image for uncertainty propagation, a bit mask image plane, and key-value metadata. They can also contain WCS and PSF model information. In the next steps you’ll learn how to display an Exposure’s image and mask.

## Create a Display

To display the `calexp` you will use the LSST `afwDisplay` framework. It provides a uniform API for multiple display backends, including DS9, matplotlib, and LSST’s Firefly viewer. The default backend is `ds9`, but since we are working remotely on `jupyterhub` we would prefer to use the web-based Firefly display. A [user guide](https://pipelines.lsst.io/v/daily/modules/lsst.display.firefly/index.html)  for `lsst.display.firefly` is available on the [pipelines.lsst.io site](https://pipelines.lsst.io/v/daily).

Now, we create a Firefly display.

In [5]:
afwDisplay.setDefaultBackend('firefly')
display1 = afwDisplay.Display(frame=1)

In [6]:
# Yusra notes that if you're on lsst-dev, you can do:

# 
# display1 = afwDisplay.Display(backend='firefly', frame=1, url='https://lsst-demo.ncsa.illinois.edu/firefly')
# display1.getClient().get_firefly_url()
#
# I've been told not to do this becaue your notebook won't be portable, but to instead do:
# export FIREFLY_URL=https://lsst-demo.ncsa.illinois.edu/firefly
#


If you're on the LSP, you can drag the window to the right side so that you can see the display side-by-side with the notebook.  Or you can embed it:

In [7]:
from IPython.display import IFrame
IFrame(display1.getClient().get_firefly_url(), 1100, 700)

## Display the calexp (calibrated exposure)

We can now build the display and use the `mtv` method to view the `calexp` with Firefly. First we display an image with mask planes and then overplot some sources.

In [8]:
display1.mtv(calexp)

As soon as you execute the command a single simulated, calibrated LSST exposure, the `{'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}` data ID, should appear in the Firefly browser window.

Notice that the image is overlaid with colorful regions. These are mask regions. Each color reflects a different mask bit that correspond to detections and different types of detector artifacts. You’ll learn how to interpret these colors later, but first you’ll likely want to adjust the image display.

## Improving the image display

The display framework gives you control over the image display to help bring out image details. For example, to make masked regions semi-transparent, so that underlying image features are visible, try:

In [9]:
display1.setMaskTransparency(80)

The setMaskTransparency method’s argument can range from 0 (fully opaque) to 100 (fully transparent).

You can also control the colorbar scaling algorithm with the display’s scale method. Try an asinh stretch with explicit minimum (black) and maximum (white) values:

In [10]:
display1.scale("asinh", -1, 30)

You can also use an automatic algorithm like `zscale` (or `minmax`) to select the white and black thresholds:

In [11]:
display1.scale("linear", "zscale")

## Interpreting displayed mask colors

The display framework renders each plane of the mask in a different color (plane being a different bit in the mask). To interpret these colors you can get a dictionary of mask planes from the `calexp` and query the display for the colors it rendered each mask plane with. For example:

In [12]:
mask = calexp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, display1.getMaskPlaneColor(maskName)))

BAD: red
CR: magenta
DETECTED: blue
DETECTED_NEGATIVE: cyan
EDGE: yellow
INTRP: green
NO_DATA: orange
SAT: green
SUSPECT: yellow


In the Firefly viewer tab, the overlays button ![overlays button](http://irsa.ipac.caltech.edu/onlinehelp/finderchart/img/layers.png) on the toolbar gives you very detailed control over the mask planes, such as turning individual planes on and off, changing the color and adjusting the transparency. Mask transparency and colors can also be set using `afw.display` commands, for individual planes or for all.

## Plotting sources on the display

The LSST processing pipeline also creates a table of the sources it used for PSF estimation as well as astrometric and photometric calibration. The dataset type of this table is `src`, which you can get from the Butler:

In [13]:
src = butler.get('src', **dataId)

The returned object, `src`, is a `lsst.afw.table.SourceTable` object. `SourceTables` are explored more elsewhere, but you can do some simple investigations using common python functions. For example, to check the length of the object:




In [14]:
len(src)

2662

You can view an HTML rendering of the `src` table by getting an `astropy.table.Table` version of it:

In [15]:
src.asAstropy()[:5]

id,coord_ra,coord_dec,parent,calib_detected,calib_psfCandidate,calib_psfUsed,calib_psf_reserved,deblend_nChild,deblend_deblendedAsPsf,deblend_psfCenter_x,deblend_psfCenter_y,deblend_psfFlux,deblend_tooManyPeaks,deblend_parentTooBig,deblend_masked,deblend_skipped,deblend_rampedTemplate,deblend_patchedTemplate,deblend_hasStrayFlux,base_GaussianCentroid_x,base_GaussianCentroid_y,base_GaussianCentroid_flag,base_GaussianCentroid_flag_noPeak,base_GaussianCentroid_flag_resetToPeak,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,base_NaiveCentroid_flag_resetToPeak,base_SdssCentroid_x,slot_Centroid_x,base_SdssCentroid_y,slot_Centroid_y,base_SdssCentroid_xSigma,base_SdssCentroid_xErr,slot_Centroid_xSigma,base_SdssCentroid_ySigma,base_SdssCentroid_yErr,slot_Centroid_ySigma,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_GaussianCentroid_flag_badInitialCentroid,base_GaussianFlux_flag_badCentroid,base_NaiveCentroid_flag_badInitialCentroid,base_PsfFlux_flag_badCentroid,base_SdssShape_flag_badCentroid,base_Variance_flag_badCentroid,ext_photometryKron_KronFlux_flag_badInitialCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,slot_Centroid_flag,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_GaussianCentroid_flag_badInitialCentroid_edge,base_GaussianFlux_flag_badCentroid_edge,base_NaiveCentroid_flag_badInitialCentroid_edge,base_PsfFlux_flag_badCentroid_edge,base_SdssShape_flag_badCentroid_edge,base_Variance_flag_badCentroid_edge,ext_photometryKron_KronFlux_flag_badInitialCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,slot_Centroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_GaussianCentroid_flag_badInitialCentroid_noSecondDerivative,base_GaussianFlux_flag_badCentroid_noSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,base_SdssShape_flag_badCentroid_noSecondDerivative,base_Variance_flag_badCentroid_noSecondDerivative,ext_photometryKron_KronFlux_flag_badInitialCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,slot_Centroid_flag_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_GaussianCentroid_flag_badInitialCentroid_almostNoSecondDerivative,base_GaussianFlux_flag_badCentroid_almostNoSecondDerivative,base_NaiveCentroid_flag_badInitialCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,base_SdssShape_flag_badCentroid_almostNoSecondDerivative,base_Variance_flag_badCentroid_almostNoSecondDerivative,ext_photometryKron_KronFlux_flag_badInitialCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmShapeRegauss_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMomentsRound_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,slot_Centroid_flag_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_GaussianCentroid_flag_badInitialCentroid_notAtMaximum,base_GaussianFlux_flag_badCentroid_notAtMaximum,base_NaiveCentroid_flag_badInitialCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notA

Now we’ll overplot sources from the `src` table onto the image display using the Display’s `dot` method for plotting markers. `Display.dot` plots markers individually, so you’ll need to iterate over rows in the `SourceTable`. Next we display the first 100 sources. We limit the number of sources since plotting the whole catalog is a serial process and takes some time. Because of this, it is more efficient to send a batch of updates to the display, so we enclose the loop in a `display.Buffering` context, like this:

In [16]:
with display1.Buffering():
    for record in src:
        display1.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

Note that the first 100 sources are preferentially located at the bottom of the image. This spatial ordering is likely imprinted by the source detection algorithm; however, it could change due to parallelization. The units o the `size` parameter are believed to be pixels.

## Clearing markers

`Display.dot` always adds new markers to the display. To clear the display of all markers, use the erase method:

In [17]:
display1.erase()

## Using FireflyClient directly.

We can also use the Firefly client directly to make plots and add catalogs to the visualization.  First retrieve the `FireflyClient` object.

In [18]:
fc = display1.getClient()

For uploading a table it is convenient to use the firefly_client.plot module. Import it and ensure it is using the same FireflyClient instance.

In [19]:
import firefly_client.plot as ffplt
ffplt.use_client(fc)

Let's select just the sources that were used to fit the PSF. 

In [20]:
psf_src = src[src['calib_psfUsed']]
print(src['calib_psfUsed'])

[False False False ... False False False]


Upload a SourceCatalog to Firefly. By default, the catalog is shown in an interactive table viewer.

In [21]:
tbl_id = ffplt.upload_table(psf_src, title='Source Catalog')

Make a scatter plot.

In [22]:
ffplt.scatter(x_col='base_CircularApertureFlux_12_0_flux/base_GaussianFlux_flux',
              y_col='log10(base_CircularApertureFlux_12_0_flux)',
              size=4,
              color='blue',
              title='test ap flux/model mag vs. log(ap flux)',
              xlabel='Model',
              ylabel='Ap/Model')

Plots are rendered by `plotly`, so follow the same syntax for construction.  For info on `plotly` see the [primer](https://plot.ly/python/getting-started/) and [examples](https://plot.ly/python/line-and-scatter/).

In [23]:
display1.erase()

## Wrap up
In this tutorial we have used the `Butler` to access LSST simulation data and used the LSST Science Pipelines Python API to display images and tables. Here are some key takeaways:

* Use the `lsst.daf.persistence.Butler` class to read and write data from repositories.
* The `lsst.afw.display` module provides a flexible framework for sending data from LSST Science Pipelines code to image displays. We used the Firefly backend for web-based visualization (`ds9`, `matplotlib`, and `ginga` are other avialable backends).
* `Exposure` objects have image data, mask data, and metadata. When you display an Exposure, the display framework automatically overlays mask planes.
* We have accessed and visualized the Table of sources extracted from an image. The `Table.asAstropy` method can be used to view the table as an `astropy.table.Table`.
* We have sent a subset of objects to directly to the Firefly API for additional plotting and investigation.

------

 # Part II  https://github.com/lsst/display_firefly/blob/master/examples/HSC-Footprints.ipynb

In [24]:
display1.erase()
butler = Butler('/datasets/hsc/repo/rerun/RC/w_2019_22/DM-19244')

# Define an ID for retrieving data

dataId = dict(filter='HSC-R', tract=9813, patch='4,4')

# Define a bounding boxes for a region of interest, one for the catalog and a larger one for the image.

footprintsBbox = afwGeom.Box2I(corner=afwGeom.Point2I(16900, 18700),
                     dimensions=afwGeom.Extent2I(600,600))
imageBbox = afwGeom.Box2I(corner=afwGeom.Point2I(16800, 18600),
                     dimensions=afwGeom.Extent2I(800,800))
                               
# Retrieve a cutout for the corresponding exposure / coadd
coadd = butler.get('deepCoadd_sub', dataId=dataId, bbox=imageBbox)

# Retrieve a catalog for the entire region
measCat = butler.get('deepCoadd_meas', dataId=dataId)


In [25]:
catSelect = np.array([footprintsBbox.contains(afwGeom.Point2I(r.getX(), r.getY()))
                       for r in measCat])
catalogSubset = measCat.subset(catSelect)

In [26]:
display1.setMaskTransparency(80)
display1.scale('asinh', 10, 80, unit='percent', Q=6)

In [27]:
display1.resetLayout()
display1.mtv(coadd)

Overlay the footprints and accompanying table.

Colors can be specified as a name like `'cyan'` or `afwDisplay.RED`; as an rgb value such as `'rgb(80,100,220)'`; or as rgb plus alpha (opacity) such as `'rgba('74,144,226,0.60)'`.

The `layerString` and `titleString` are concatenated with the frame, to make the footprint drawing layer name and the table title, respectively. If multiple footprint layers are desired, be sure to use different values of `layerString`.

In [28]:
display1.overlayFootprints(catalogSubset, color='rgba(74,144,226,0.50)',
                           highlightColor='yellow', selectColor='orange',
                           style='outline', layerString='detection footprints ',
                           titleString='catalog footprints ')

_YA: Firefly can do a lot that's not exposed yet, because we don't have the interface with the stack._

This was quickly written as a dmeo using `deepCoadd_meas` files which have had measurement and the deblender run. It would also be useful to be able to pass in a `footprint` detection catalog rather than a measurement catalog:

Gregory says: "A PR would be most welcome"

`https://github.com/lsst/display_firefly/blob/master/python/lsst/display/firefly/footprints.py`

## Part III Fun with HSC image differences

In [29]:
import lsst.daf.persistence as dafPersist
import sqlite3
import pandas as pd
REPO = '/datasets/hsc/repo/rerun/private/yusra/RC2/ap_pipe_PDR1_bestcase'

# Connect to the PPDB
connection = sqlite3.connect(f'{REPO}/association.db')
# Make a butler
butler = dafPersist.Butler(REPO)

In [30]:
# You can use the PPDB:

ds = pd.read_sql_query("""select ccdVisitId, ra, decl, psFlux, psFluxErr, midPointTai, 
x, y, ixxPSF, iyyPSF, ixyPSF, flags, snr, apFlux, apFluxErr, 
dipMeanFlux, dipMeanFluxErr, dipFluxDiff, 
dipFluxDiffErr, dipMeanFlux_dipFluxDiff_Cov, 
totFlux, totFluxErr, diffFlux, diffFluxErr 
from DiaSource where filterId=4""", connection)

In [31]:
# create some derived columns

ds['visit'] = ds.ccdVisitId.apply(lambda x: int(x/200.))
ds['ccd'] = ds.ccdVisitId.apply(lambda x: int(np.round((x/200. - int(x/200.))*200)))

SIGMA2FWHM = 2*np.sqrt(2*np.log(2))
PIXEL_SCALE = 0.168
traceRadius = np.sqrt(0.5) * np.sqrt(ds.ixxPSF + ds.iyyPSF)
ds['seeing'] = SIGMA2FWHM * traceRadius

In [32]:
# Get flags
from lsst.ap.association import UnpackPpdbFlags, MapDiaSourceConfig
config = MapDiaSourceConfig()
unpacker = UnpackPpdbFlags(config.flagMap, 'DiaSource')
flag_values = unpacker.unpack(ds['flags'], 'flags')
flag_names = list(flag_values.dtype.names)
flagDF = pd.DataFrame(flag_values)
flagDF.head()

/opt/lsst/software/stack/stack/miniconda3-4.5.12-1172c30/Linux64/ap_association/17.0.1-6-ga2de75c+7/python/lsst/ap/association/mapApData.py:388: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  table_list = list(yaml.load_all(yaml_stream))


,base_PixelFlags_flag,base_PixelFlags_flag_offimage,base_PixelFlags_flag_edge,base_PixelFlags_flag_interpolated,base_PixelFlags_flag_saturated,base_PixelFlags_flag_cr,base_PixelFlags_flag_bad,base_PixelFlags_flag_suspect,base_PixelFlags_flag_interpolatedCenter,base_PixelFlags_flag_saturatedCenter,...,slot_PsfFlux_flag_edge,ip_diffim_forced_PsfFlux_flag,ip_diffim_forced_PsfFlux_flag_noGoodPixels,ip_diffim_forced_PsfFlux_flag_edge,slot_Shape_flag,slot_Shape_flag_unweightedBad,slot_Shape_flag_unweighted,slot_Shape_flag_shift,slot_Shape_flag_maxIter,slot_Shape_flag_psf
0,False,False,True,True,True,True,True,False,True,True,...,False,False,False,False,True,True,False,False,False,False
1,False,False,True,True,False,True,True,False,True,False,...,False,False,False,False,True,True,False,False,False,False
2,False,False,True,True,False,True,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False


In [33]:
# Bad Flag Names are our current recommendation filtering difference image detections
badFlagNames = ['base_PixelFlags_flag_saturated', 
                'base_PixelFlags_flag_suspect',
                'base_PixelFlags_flag_bad',
                'slot_Shape_flag']

In [34]:
idxGood = flagDF[badFlagNames].sum(axis = 1) == 0 
idxBad =  flagDF[badFlagNames].sum(axis = 1) != 0 

In [35]:
dataId = dict(ccd=58, visit=19658)
exp = butler.get('deepDiff_differenceExp', ccd=58, visit=19658)
cat = butler.get('deepDiff_diaSrc', dataId=dataId)

In [36]:
# You can reinitialize a Firefly viewer tab with
display1.clearViewer()
# Making a display tab reopen after closing it
display1.show()


In [37]:
display1.mtv(exp)

In [38]:
# Plot unflagged sources:
subset = ds[(ds.visit==19658) & (ds.ccd==58) & (idxGood)]

with display1.Buffering():
    for key, record in subset.iterrows():
        display1.dot('o', record['x'], record['y'], size=20, ctype='cyan')

It gets overwhelmed if you plot thousands of dots on an image, so we downsample the rows with shape flags and the large footprint peaks:

In [39]:
# Plot flagged sources:
subset = ds[(ds.visit==19658) & (ds.ccd==58) & (idxBad)]

print(len(subset))
# plot only a random 500 rows 
with display1.Buffering():
    for key, record in subset.sample(500).iterrows():
        display1.dot('+', record['x'], record['y'], size=20, ctype='red')

3446


In [40]:
# Plot peaks in larger footprints
with display1.Buffering():
    for record in cat:
        footprint = record.getFootprint()
        if footprint.getArea() > 1000:
            # Plot only every 10th peak. 
            for peak in footprint.getPeaks()[::10]:
                display1.dot('+', peak['f_x'], peak['f_y'], size=10, ctype='orange')